In [1]:
import import_ipynb
import os
from webdriver_manager.chrome import ChromeDriverManager
from category_list_clean import category_clean_data
from channel_detail_csv import channel_details_csv
from scrape_code import top_youtuber
from get_info_youtube import get_user_info
from get_top_youtube_info import get_top_user_info

importing Jupyter notebook from category_list_clean.ipynb
importing Jupyter notebook from channel_detail_csv.ipynb
importing Jupyter notebook from scrape_code.ipynb
importing Jupyter notebook from get_info_youtube.ipynb
importing Jupyter notebook from get_top_youtube_info.ipynb


In [ ]:
#Calling code to retrieve the cleaned category list
category_list = category_clean_data()

category_list

In [ ]:
#Channel details from csv file
channel_info =channel_details_csv()
channel_info

In [ ]:
#Top channels scrapped from website using selenium & beautiful soup
user_details = top_youtuber()
user_details

In [ ]:
#Retreive more information on channels from csv file through youtube API
channel_detailed_info_csv = get_user_info()
channel_detailed_info_csv

In [2]:
#Retreive more information on channels from scrapped through youtube API
top_channel_detailed_info = get_top_user_info()
top_channel_detailed_info

completed 0
Details of 1 channels are extracted successfully
completed 1
Details of 2 channels are extracted successfully
completed 2
Details of 3 channels are extracted successfully
completed 3
Details of 4 channels are extracted successfully
completed 4
Details of 5 channels are extracted successfully
'country'
Details of 5 channels are extracted successfully
completed 5
Details of 6 channels are extracted successfully
completed 6
Details of 7 channels are extracted successfully
completed 7
Details of 8 channels are extracted successfully
completed 8
Details of 9 channels are extracted successfully
completed 9
Details of 10 channels are extracted successfully
completed 10
Details of 11 channels are extracted successfully
completed 11
Details of 12 channels are extracted successfully
completed 12
Details of 13 channels are extracted successfully
completed 13
Details of 14 channels are extracted successfully
completed 14
Details of 15 channels are extracted successfully
completed 15
De

,Channel_id,Channel_title,Created_date,Video_count,Views_count,Subscription_count,Subscription_count_hidden,Country
0,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,2006-03-13T14:27:05Z,15276,151555112010,180000000,False,IN
1,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery Rhymes,2006-09-01T22:18:49Z,629,99306304600,109000000,False,US
2,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,2010-04-29T10:54:00Z,4313,27179058415,109000000,False,US
3,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,2006-09-20T22:24:59Z,54896,85211034290,102000000,False,IN
4,UCk8GzjMOrta8yxDcKfylJYw,✿ Kids Diana Show,2015-05-12T08:26:57Z,847,54914440601,77400000,False,US
...,...,...,...,...,...,...,...,...
75,UCQ7x25F6YXY9DvGeHFxLhRQ,shfa2 - شفا,2017-11-06T23:18:50Z,891,16431228088,28400000,False,AE
76,UClZF7OsrECe3t_ADrjZEh_Q,Geet MP3,2016-10-22T03:37:32Z,419,9154217030,28400000,False,IN
77,UCbrd1vu4_7qIE6IPV_dA-OA,GENIAL,2017-06-01T07:07:20Z,4075,5145602333,28400000,False,MX
78,UCtW7qWjpCZ8zps-Cf2NF26w,Gaane Sune Ansune,2015-08-24T08:17:50Z,2602,13551796339,28200000,False,IN


In [3]:
top_channel_detailed_info.to_csv('top_user.csv')